In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from ephesus.sentence import *
from sklearn.model_selection import train_test_split

In [19]:
data = extract_json("PROJECT_EPHESUS-labeling_V01.json")

In [20]:
dico = tune_data(data)

In [14]:
dico["annotations"][0]

{'text': 'Donc vaccin antigrippale, domicile non facturés.',
 'entities': [(5, 24, 'Treatment'), (26, 34, 'Location')]}

In [21]:
create_training_data(dico,"training_dataV2.spacy")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:02<00:00, 108.92it/s]

training data created under '../raw_data/training_dataV2.spacy'


# test model

In [1]:
!pwd

/home/thomas/code/GeoffroyGit/ephesus/notebooks


In [3]:
import spacy

/home/thomas/.pyenv/versions/3.8.12/envs/ephesus/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
nlp_ner = spacy.load("../models/model-best")

In [26]:
doc = nlp_ner("Prise de sang réalisée ce matin")
spacy.displacy.render(doc, style="ent",jupyter=True)

# Tuto from https://www.kaggle.com/code/finalepoch/medical-ner-using-spacy

In [27]:
from __future__ import unicode_literals, print_function
import random
from pathlib import Path
from spacy.util import minibatch, compounding
import spacy
import sys